In [ ]:
pip install opencv-python ultralytics

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import os
import re

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
results = model.track(cv2.imread('./MOT17/frames/frame215.jpg'), persist=True, classes=[41])


0: 384x640 7 cups, 87.4ms
Speed: 2.1ms preprocess, 87.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


# Cup

In [ ]:
paths = [f for f in os.listdir('./MOT17/frames') if f.endswith(('.jpg', '.png', '.jpeg'))]
paths = sorted(paths, key=lambda s: [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)])
paths = [f"./MOT17/frames/{p}" for p in paths]

In [ ]:
def readFiles(paths) :
  images = []
  for i in paths :
    images.append(cv2.imread(i))
  return images

In [ ]:
def parseResults(path, result, target_cl) :
  valid_results = []
  cls = result.cls.numpy()
  for idx,cl in enumerate(cls) :
    if cl == target_cl :
      x,y = result.xyxy[idx].numpy()[0:2]
      w,h = result.xywh[idx].numpy()[2:4]
      id = result.id[idx].item()
      frame_num = path.split("e")[-1]
      frame_num = int(frame_num.split('.')[0])
      valid_results.append([frame_num,id,x,y,w,h])
  return valid_results

In [ ]:
def track(paths,imgs, target_cl = 41.0) :
  res_arr = []
  for idx,im in enumerate(imgs) :
    results = model.track(im, persist=True, verbose=False)
    results = model.track(im, persist=True, verbose=False)
    res_arr += parseResults(paths[idx],results[0].boxes,target_cl)
  return res_arr

In [ ]:
def buildString(frame_tuple) :
  str_arr = list(map(str,np.array(frame_tuple).astype(int)))
  return " ".join(str_arr)

In [ ]:
imgs = readFiles(paths)

In [ ]:
res = track(paths,imgs)

In [ ]:
for r in res :
   s = buildString(r)
   with open("./result.txt", 'a+') as file:
             file.write(s + "\n")

#TrackEval

In [ ]:
!git clone https://github.com/SlimaneBouss/MOT17.git

In [ ]:
!git clone https://github.com/JonathonLuiten/TrackEval.git

In [ ]:
!wget https://omnomnom.vision.rwth-aachen.de/data/TrackEval/data.zip
!unzip data.zip -d TrackEval

In [ ]:
mot17_folder = "MOT17/MOT17/train"
directories = [d for d in os.listdir(mot17_folder)]
directories.sort()
directories.pop(0)


for dir in directories:


    images_path = f"{mot17_folder}/{dir}/img1"
    output_file = f"./TrackEval/data/trackers/mot_challenge/MOT17-train/DeepSort/data/{dir}.txt"

    image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

    # Sort the files by their digits, take off the word "frame" and the extension
    image_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    # Load the YOLOv8 model detection
    model = YOLO('yolov8n.pt')

    for image_id, image in enumerate(image_files):

        image = cv2.imread(os.path.join(images_path, image_files[image_id]))
        image_id += 1

        # Run YOLOv8 tracking on the frame, persisting tracks between frames, and select class pedestrian
        results = model.track(image, imgsz=640, data='coco128.yaml', persist=True, classes=[0], verbose=False)

        # Get the boxes and track IDs
        if results[0].boxes.id is not None and hasattr(results[0].boxes, 'id'):
            boxes = results[0].boxes.xyxy.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Ajouter les détections au dictionnaire detections_dict
            for box, track_id in zip(boxes, track_ids):
                x_min = box[0]
                y_min = box[1]
                x_max = box[2]
                y_max = box[3]
                width = x_max - x_min
                height = y_max - y_min

                with open(output_file, 'a+') as file:
                    file.write(f"{image_id},{track_id},{x_min},{y_min},{width},{height},{1},{-1},{-1},{-1} \n")


In [ ]:
!python TrackEval/scripts/run_mot_challenge.py  --TRACKERS_TO_EVAL DeepSort --METRICS HOTA --USE_PARALLEL False

